In [28]:
# Import libraries
import pandas as pd
import numpy as np
import sqlite3
import requests 
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
import plotly.express as px
import matplotlib.pyplot as plt

In [29]:
# Build connection
connection=sqlite3.connect('../input/california-traffic-collision-data-from-switrs/switrs.sqlite')

In [30]:
pip freeze

In [31]:
# Display columns of collisions table
query = "SELECT collision_time, collision_date, county_location, county_city_location, collision_severity, killed_victims, injured_victims FROM collisions"
collisions = pd.read_sql_query(query,connection)

In [32]:
# Display table
collisions.head()

In [33]:
# Display info
collisions.info()

In [34]:
# Display nulls
collisions.isna().sum()

In [35]:
# Drop nulls in collision_time column and fill other nulls with 0
collisions = collisions.dropna(subset=["collision_time"])
collisions = collisions.fillna(0)

In [36]:
# Display nulls
collisions.isna().sum()

In [37]:
# Display unique values under the collision_severity column
collisions["collision_severity"].unique()

In [38]:
# Display unique values under the county_location column
np.sort(collisions["county_location"].unique())

In [39]:
# Display unique values under the county_city_location column
collisions["county_city_location"].unique()

In [40]:
# Convert to pandas datetime
collisions["collision_date"] = pd.to_datetime(collisions["collision_date"])

In [41]:
# Get FIPS codes of counties
url = "https://en.wikipedia.org/wiki/List_of_counties_in_California#List"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(url)

# Parse table
soup = BeautifulSoup(response.text, 'html.parser')
california_counties=soup.find('table',{'class':"wikitable"})

# Load table to pandas df
california_counties=pd.read_html(str(california_counties))
california_counties=pd.DataFrame(california_counties[0])

# Select columns
california_counties = california_counties[["County", "FIPS code[5]"]]

# Rename columns
california_counties = california_counties.rename(
    columns={
        'County': 'county', 
        'FIPS code[5]': 'FIPS'
    }
)

# Convert FIPS column to string
california_counties["FIPS"] = california_counties["FIPS"].astype('str')

# Convert column to plotly-compatible FIPS format
california_counties["FIPS"] = california_counties["FIPS"].apply(lambda x: x.zfill(3))
california_counties["FIPS"] = "06"+california_counties["FIPS"]

# Lowercase and remove county word in county column
california_counties["county"]=california_counties['county'].str.lower()
california_counties["county"] = california_counties["county"].str.replace(
    " county",
    ""
)

In [42]:
# Show data
california_counties

In [43]:
# Show data info
collisions.info()

In [44]:
# Rename county_location
collisions = collisions.rename(
    columns={
        'county_location': 'county'
    }
)

In [45]:
# Combine collisions and california_counties
collisions = pd.merge(collisions, california_counties, on='county')

In [46]:
# Show data info
collisions.info()

In [47]:
# Create hour, month and year columns
collisions["collision_hour"] = pd.to_datetime(collisions["collision_time"]).dt.strftime('%H').astype(int)
collisions["collision_month"] = collisions["collision_date"].dt.month
collisions["collision_year"] = collisions["collision_date"].dt.year
collisions["collision_day"] = collisions["collision_date"].dt.dayofweek

In [48]:
# Create list of year options
year_options = ["all"]
year_list = range(2001, 2022)
for year in year_list:
    year_options.append(year)

In [49]:
# Create choropleth df
choropleth_df = pd.DataFrame()

for year in year_options:
    if year == "all":
        temp = collisions.groupby(["county", "FIPS"])["killed_victims"].sum().reset_index()
    else:
        year = int(year)
        temp = collisions[collisions["collision_year"] == year]
        temp = temp.groupby(["county", "FIPS"])["killed_victims"].sum().reset_index()
    
    # Add identification column
    temp["year_option"] = year
    
    # Concat df
    frames = [choropleth_df, temp]
    choropleth_df = pd.concat(frames,ignore_index=True)

In [50]:
# Show choropleth_df
choropleth_df

In [51]:
# Save choropleth_df
choropleth_df.to_csv("choropleth.csv")

In [52]:
# Create hourly df
hourly_df = pd.DataFrame()

for year in year_options:
    if year == "all":
        temp = collisions.groupby(["collision_hour"])["killed_victims"].sum().reset_index()
    else:
        year = int(year)
        temp = collisions[collisions["collision_year"] == year]
        temp = temp.groupby(["collision_hour"])["killed_victims"].sum().reset_index()
        
    # Add identification column
    temp["year_option"] = year
    
    # Concat df
    frames = [hourly_df, temp]
    hourly_df = pd.concat(frames,ignore_index=True)

In [53]:
# Show hourly_df
hourly_df

In [54]:
# Save hourly_df
hourly_df.to_csv("hourly.csv")

In [55]:
# Create day_of_week df
day_of_week_df = pd.DataFrame()

for year in year_options:
    if year == "all":
        temp = collisions.groupby(["collision_day"])["killed_victims"].sum().reset_index()
    else:
        year = int(year)
        temp = collisions[collisions["collision_year"] == year]
        temp = temp.groupby(["collision_day"])["killed_victims"].sum().reset_index()
    
    # Preprocess the data
    temp = temp.sort_values(by=['collision_day'], ascending=True)
    days = {0: 'Mon', 1: 'Tues', 2: 'Weds', 3: 'Thurs', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
    temp['collision_day'] = temp['collision_day'].apply(lambda x: days[x])
    
    # Add identification column
    temp["year_option"] = year
    
    # Concat df
    frames = [day_of_week_df, temp]
    day_of_week_df = pd.concat(frames,ignore_index=True)

In [56]:
# Show day_of_week_df
day_of_week_df

In [57]:
# Save day_of_week_df
day_of_week_df.to_csv("day_of_week.csv")

In [58]:
# Save file
collisions.to_csv("collisions.csv")